<a href="https://colab.research.google.com/github/digodox/Imersao_Alura_Google_AI/blob/main/Projeto_Imersao_Alura_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importação das bibliotecas

In [68]:
import hashlib
from IPython.display import display
from IPython.display import Markdown
import textwrap
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

##Configurando a API Key

In [69]:
api_key = userdata.get("GENAI_API_KEY")
genai.configure(api_key=api_key)

##Configuração do modelo

In [70]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

system_instruction = "Aja como um engenheiro."

##Markdown

In [71]:
def to_markdown(text):
  text = text.replace("'", " *")
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


##Extração dos dados das planilhas e preparando o histórico do chat

In [72]:
def extrair_dados_racionalizacao(alarme):
  tagname = alarme['Tagname']
  descricao = alarme['Descrição do Alarme']
  prioridade = alarme['Prioridade']
  causa = alarme['Causa iniciadora']
  acao = alarme['Ação da Central de Controle']
  impacto = alarme['Impacto direto']
  consequencia = alarme['Consequência Potencial']
  supressao = alarme['Supressão Manual']

  dado_extraido = f"O alarme '{tagname}', que se refere à '{descricao}', e considerando que a causa iniciadora é a '{causa}', e com prioridade '{prioridade}'."
  return dado_extraido

def extrair_dados_alarmes(alarme):
  tagname = alarme['Tagname']
  status = alarme['Status do Alarme']

  dado_extraido = f"O alarme '{tagname}' está no estado '{status}'."
  return dado_extraido

def extrair_dados_valores_anal(alarme):
  tagname = alarme['TagName']
  description = alarme['Description']
  value = alarme['Value']
  units = alarme['Units']

  dado_extraido = f"O tag '{tagname}', que se refere à '{description}', possui o valor atual de '{value}' '{units}'."
  return dado_extraido

def extrair_dados_limites_operacionais(alarme):
  estacao = alarme['Estação']
  variavel = alarme['Variável']
  limite_superior = alarme['Limite Superior']
  limite_inferior = alarme['Limite Inferior']
  units = alarme['Units']

  dado_extraido = f"A variável '{variavel}', que se refere à estação '{estacao}', possui limite máximo de '{limite_superior}' '{units}' e limite mínimo de '{limite_inferior}' '{units}'."
  return dado_extraido

##Definição do contexto inicial na variável history[]

In [73]:
#Chat history variable
history = []

df = pd.read_excel("/content/sample_data/Dados/Racionalizacao_de_alarmes.xlsx")
for index, row in df.iterrows():
  alarme = row.to_dict()
  dados_racionalizacao = extrair_dados_racionalizacao(alarme)
  history.append({
      "role": "user",
      "parts": dados_racionalizacao
      })

df = pd.read_excel("/content/sample_data/Dados/Alarmes.xlsx")
for index, row in df.iterrows():
  alarme = row.to_dict()
  dados_alarmes = extrair_dados_alarmes(alarme)
  history.append({
      "role": "user",
      "parts": dados_alarmes
      })

df = pd.read_excel("/content/sample_data/Dados/TAGs_Analogicos.xlsx")
for index, row in df.iterrows():
  alarme = row.to_dict()
  dados_tags_analogicos = extrair_dados_valores_anal(alarme)
  history.append({
    "role": "user",
    "parts": dados_tags_analogicos
    })

df = pd.read_excel("/content/sample_data/Dados/Limites Operacionais.xlsx")
for index, row in df.iterrows():
  alarme = row.to_dict()
  dados_limites_operacionais = extrair_dados_limites_operacionais(alarme)
  history.append({
    "role": "user",
    "parts": dados_limites_operacionais
    })

##Inicialização do modelo

In [74]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

## Chat prompt

Sugestão de perguntas para o modelo:

*   Sumarize o status das estações.
*   Qual o status da estação 01.
*   Quais ações tomar sobre o alarme crítico da estação 01.
*   Digite "fim" no prompt para encerrar o chat.

In [75]:
convo = model.start_chat(history=history)

prompt = input("Digite sua pergunta: ")
while prompt != "fim":
  convo.send_message(prompt)
  print("Resposta: ")
  display(to_markdown(convo.last.text))
  print("\n")
  prompt = input("Digite sua pergunta: ")


Digite sua pergunta: Sumarize o status das estações
Resposta: 


> ## Sumário do Status das Estações CITY
> 
> **Alarmes:**
> 
> * **CITY_05_LSL-053B:** Nível de água no aquecedor B - Atuado. Possível falta de água na caixa d *água da estação ou problema no instrumento. Prioridade BAIXA.
> * **CITY_01_XS-056A-FALHA:** Válvula de bloqueio na entrada do aquecedor A - Falha.  Falha no comando da válvula HV-056. Prioridade CRÍTICA.
> * **CITY_02_ZSH-410:** Válvula VES-410 Aberta. Confirmação do comando de abertura, falha do instrumento ou abertura indevida. Prioridade CRÍTICA.
> * **CITY_07_ZSL-014B:** Válvula XV-014B Fechada. Falha do equipamento, falha dos tramos de redução, atuação ou indicação indevida. Prioridade ALTA.
> 
> **Variáveis fora dos limites:**
> 
> * **CITY_02_GFC-qb-ABC:** Vazão volumétrica total (411.56 km³/d) acima do limite máximo (100 km³/d)
> * **CITY_02_PT-033:** Pressão na saída da estação (49.83 kgf/cm²) acima do limite máximo (35 kgf/cm²)
> * **CITY_05_PT-033:** Pressão na saída da estação (42.43 kgf/cm²) acima do limite máximo (35 kgf/cm²)
> 
> **Observações:**
> 
> * Todas as outras estações apresentam status normal para os alarmes e variáveis monitorados.
> * As estações CITY_02 e CITY_05 apresentam pressão de saída acima do limite, requerendo investigação e possível intervenção.
> * A estação CITY_02 apresenta vazão volumétrica total muito acima do limite, demandando atenção imediata para identificar a causa e tomar as medidas corretivas.
> * A falha crítica na válvula de bloqueio da entrada do aquecedor A na estação CITY_01 precisa ser resolvida com urgência.
> 
> Recomenda-se  investigar as causas dos alarmes e variáveis fora dos limites operacionais, priorizando as falhas críticas e tomando as medidas corretivas adequadas. 




Digite sua pergunta: Qual o status da estação 01
Resposta: 


> ## Status da Estação CITY_01:
> 
> **Alarmes:**
> 
> * **CITY_01_XS-056A-FALHA:** Válvula de bloqueio na entrada do aquecedor A - Falha. Falha no comando da válvula HV-056. Prioridade CRÍTICA.
> 
> **Variáveis:**
> 
> * **CITY_01_GFC-qb-AB:** Vazão volumétrica total: 25.59 km³/d (Dentro dos limites)
> * **CITY_01_PT-001:** Pressão de entrada da estação: 82.05 kgf/cm² (Dentro dos limites)
> * **CITY_01_PT-033:** Pressão na saída da estação: 35.31 kgf/cm² (Dentro dos limites)
> 
> **Observações:**
> 
> * A estação CITY_01 apresenta um alarme crítico: falha na válvula de bloqueio na entrada do aquecedor A. Essa falha precisa ser investigada e corrigida com urgência, pois pode comprometer o funcionamento do aquecedor e impactar o abastecimento de água.
> * As demais variáveis da estação, como vazão e pressão, estão dentro dos limites operacionais, indicando que, excetuando a falha crítica, a estação opera dentro dos parâmetros esperados.
> 
> **Recomendações:**
> 
> * Investigar a causa da falha na válvula de bloqueio na entrada do aquecedor A (CITY_01_XS-056A-FALHA) e tomar as medidas corretivas o mais rápido possível. 
> * Monitorar as demais variáveis da estação para garantir que continuem dentro dos limites operacionais. 




Digite sua pergunta: Quais ações tomar sobre o alarme crítico da estação 01
Resposta: 


> ## Ações para o Alarme Crítico CITY_01_XS-056A-FALHA:
> 
> O alarme  *CITY_01_XS-056A-FALHA *, que indica falha na válvula de bloqueio na entrada do aquecedor A da estação CITY_01,  é crítico e requer ação imediata para garantir a segurança e o bom funcionamento do sistema.
> 
> **Passo 1: Confirmação e avaliação:**
> 
> 1. **Confirmar o status do alarme:** Verificar no sistema de supervisão se o alarme ainda está ativo e se há outras informações relevantes.
> 2. **Verificar a posição da válvula HV-056:**  Se possível, confirmar a posição real da válvula através de inspeção visual local ou sistema de monitoramento.
> 3. **Analisar o histórico de alarmes:** Verificar se esse alarme é recorrente e se há outros alarmes relacionados, como os relacionados ao aquecedor A.
> 
> **Passo 2: Isolamento e segurança:**
> 
> 1. **Isolar o aquecedor A:** Se a válvula HV-056 estiver realmente aberta e não responder a comandos, isole o aquecedor A para evitar possíveis danos ou riscos à segurança.
> 2. **Informar a equipe de manutenção:** Comunicar a equipe de manutenção sobre o alarme crítico e as medidas de isolamento tomadas.
> 3. **Monitorar o sistema:** Observar atentamente o comportamento do sistema, como temperatura e pressão, durante o processo de isolamento.
> 
> **Passo 3: Diagnóstico e reparo:**
> 
> 1. **Investigar a causa da falha:** A equipe de manutenção deve investigar a causa da falha na válvula HV-056. Algumas possibilidades são:
>     * **Falha no atuador da válvula:** Verificar o fornecimento de energia ao atuador, fiação e possíveis defeitos no mecanismo.
>     * **Obstrução na válvula:**  Inspecionar a válvula em busca de obstruções que impeçam o seu fechamento.
>     * **Problema no sinal de controle:** Verificar a integridade do cabeamento e a correta emissão do sinal de controle pelo sistema de automação.
> 2. **Realizar o reparo:** Após identificar a causa da falha, a equipe de manutenção deve realizar o reparo ou substituição dos componentes defeituosos.
> 
> **Passo 4:  Normalização e acompanhamento:**
> 
> 1. **Testar a válvula:**  Após o reparo, testar o funcionamento da válvula HV-056, garantindo que ela abra e feche corretamente em resposta aos comandos do sistema de automação.
> 2. **Reativar o aquecedor A:** Após a confirmação do bom funcionamento da válvula, reativar o aquecedor A.
> 3. **Monitorar o sistema:** Continuar monitorando o sistema para garantir que o alarme não volte a ocorrer e que o aquecedor A opere normalmente.
> 
> **Documentação:**
> 
> * Registrar todas as ações tomadas, os resultados das investigações e os reparos realizados.
> 
> Lembre-se que a segurança é a prioridade máxima. Se não se sentir confortável para realizar algum dos passos, contate um especialista ou a equipe de manutenção imediatamente.
> 




Digite sua pergunta: fim
